| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | array (N,) | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Nx, Ny) | X-meshgrid|
| Y | array (Nx, Ny) | Y-meshgrid|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Nx, Ny) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| trajectory | array (2,N) | integrated trajectory |
| velocity | array (2,N) | velocity along trajectory |
| DF | array (2,2,N) | gradient of flow map at every time-step |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import package to solve ODE
from scipy.integrate import solve_ivp

# function which evalutes derivative of flow map and derivative of gradient of flow map
from ipynb.fs.defs.dFdt_eqVar import dFdt_eqVar

In [3]:
def integration_dFdt_eqVar(time, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid):
    
    # initial condition [x0, DF0]
    x_init_extended = np.array([x[0], x[1], 1, 0, 0, 1])
        
    # time-interval [t0,tN] over which to integrate
    time_interval = [time[0], time[-1]]
        
    # time-array at which trajectory must be evaluated
    t_eval = time.copy()
        
    # Step-size
    dt = time[1]-time[0]
                    
    sol = solve_ivp(dFdt_eqVar, time_interval, x_init_extended, 'RK45', t_eval, rtol=1e-7, atol=1e-7, first_step = dt, args = (X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid))
          
    # Initialize velocity array
    dFdt = np.zeros((2, len(time)))
    
    # Initialize DF
    DF = np.zeros((2, 2, len(time)))
    DF[0,0,:] = sol.y[2,:]
    DF[0,1,:] = sol.y[3,:]
    DF[1,0,:] = sol.y[4,:]
    DF[1,1,:] = sol.y[5,:]
            
    for i in range(sol.y.shape[1]-1):
                
        dFdt[0, i] = (sol.y[0,i+1]-sol.y[0,i])/dt
        dFdt[1, i] = (sol.y[1,i+1]-sol.y[1,i])/dt
        
    Fmap = sol.y[:2,:].copy()
    
    if periodic[0]:
        
        Fmap[0,:] = Fmap[0,:]%(X[0, -1]-X[0, 0])
    
    if periodic[1]:
        
        Fmap[1,:] = Fmap[1,:]%(Y[-1, 0]-Y[0, 0])
            
    return Fmap, dFdt, DF